# API ROCrate Tests

In [ ]:
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder

root_url = "http://localhost:8080/"

## Create users

In [ ]:
user1_data = {
    "@id": "ark:99999/test-user1",
    "name": "Test User1",
    "type": "Person",
    "email": "testuser1@example.org",
    "password": "test1",
    "organizations": [],
    "projects": [],
    "datasets": [],
    "rocrates": [],
    "software": [],
    "computations": [],
    "evidencegraphs": []
}
user1_create = requests.post(root_url + "user", json=user1_data)
user1_create.json()


In [ ]:
# Get a user
user1_get = requests.get(root_url + "user/"+ user1_data['@id'])
user1_get.json()

## Create Organization

In [ ]:
org_data = {
    "@id": "ark:99999/test-org",
    "name": "test organization",
    "@type": "Organization",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

In [ ]:
import base64
encoded_credentials = base64.b64encode(
    bytes(f"{user1_data.get('email')}:{user1_data.get('password')}", "utf-8"))

owner_headers = {"Authorization" : f"Basic {str(encoded_credentials, 'utf-8')}" }

organization_create = requests.post(root_url + "organization", data=json.dumps(org_data), headers=owner_headers)
organization_create.json()

In [ ]:
# Get an organization
org_get = requests.get(root_url + "organization/"+ org_data['@id'], headers=owner_headers)
org_get.json()

## Create Project

In [ ]:
project_data = {
    "@id": "ark:99999/test-org/test-proj",
    "name": "Test Project",
    "@type": "Project",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    },
    "memberOf": {
        "@id": org_data['@id'],
        "name": org_data['name'],
        "@type": org_data['@type']
    }
}
project_create = requests.post(root_url + "project", json=project_data, headers=owner_headers)
project_create.json()

In [ ]:
# Get a project
project_get = requests.get(root_url + "project/"+ project_data['@id'], headers=owner_headers)
project_get.json()

## Transfer ROCrate to MinIO

In [18]:
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder

root_url = "http://localhost:8080/"
mp_encoder = MultipartEncoder(
    fields={        
        # 'file' should be same as the argument in the POST method
        'file': ('test-rocrate', open('/home/sadnan/PycharmProjects/mds_python/tests/guid_test_rocrate.zip', 'rb'), 'application/zip')
        #'file': ('test-rocrate', open('/home/sadnan/Downloads/empty-zip-file.zip', 'rb'), 'application/zip')
        #'file': ('test-rocrate', open('/home/sadnan/Downloads/sample.zip', 'rb'), 'application/zip')
        #'file': ('test-rocrate', open('/home/sadnan/Downloads/bad-zip-file.zip', 'rb'), 'application/zip')
        #'file': ('test-rocrate', open('/home/sadnan/Downloads/no-rocrate-metadata.zip', 'rb'), 'application/zip')
        #'file': ('test-rocrate', open('/home/sadnan/Downloads/extra_metadata_object_in_rocrate.zip', 'rb'), 'application/zip')


    }
)

# upload a rocrate to minio object store
rocrate_transfer = requests.post(
    root_url + f"rocrate/upload",
    data=mp_encoder,                              
    # The MultipartEncoder provides the content-type header with the boundary:
    headers={'Content-Type': mp_encoder.content_type}
)
rocrate_transfer.json()

{'created': {'@id': 'crate.guid', '@type': 'ROCrate', 'name': 'crate.name'}}

# List RO-Crates

In [19]:
list_rocrates = requests.get(root_url + "rocrate")
list_rocrates.json()

{'rocrates': [{'@id': 'ark:/UVA/B2AI/test_rocrate',
   '@type': 'Dataset',
   'name': 'test rocrate'}]}

# Download ROCrate from its identifier


download_rocrate = requests.get(root_url + f"rocrate/download/ark:99999/UVA/B2AI/test_rocrate")
download_download.json()